# [Question Answering with Hugging Face Transformers](https://keras.io/examples/nlp/question_answering/)

In [ ]:
!pip install transformers
!pip install datasets
!pip install huggingface-hub

In [ ]:
pip install layer-sdk -qqq

In [1]:
import layer
from layer.decorators import model, resources, pip_requirements, fabric, dataset

In [2]:
# layer.logout()
# layer.login("https://app.layer.ai/")
layer.login("https://development.layer.co/")

In [3]:
layer.init("qas")

In [ ]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a
    # stride. This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous
    # feature.
    from transformers import AutoTokenizer
    model_checkpoint = "distilbert-base-uncased"

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    max_length = 384  # The maximum length of a feature (question and context)
    doc_stride = (
        128  # The authorized overlap between two part of the context when splitting
    )
# it is needed.
    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a
    # map from a feature to its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original
    # context. This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what
        # is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this
        # span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the
            # CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the
                # answer.
                # Note: we could go after the last offset if the answer is the last word (edge
                # case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
@fabric("f-medium")
@pip_requirements(packages=["transformers","tensorflow","keras","sentencepiece","datasets"])
@model("qa")
def train():
    import tensorflow as tf
    from tensorflow import keras
    from transformers import TFAutoModelForQuestionAnswering
    from datasets import load_dataset
    datasets = load_dataset("squad")
    tokenized_datasets = datasets.map(prepare_train_features,batched=True,remove_columns=datasets["train"].column_names,num_proc=3,)
    train_set = tokenized_datasets["train"].with_format("numpy")[:]
    # Load the whole dataset as a dict of numpy arrays
    validation_set = tokenized_datasets["validation"].with_format("numpy")[:]
    model_checkpoint = "distilbert-base-uncased"
    model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
    optimizer = keras.optimizers.Adam(learning_rate=5e-5)
  # Optionally uncomment the next line for float16 training
    keras.mixed_precision.set_global_policy("mixed_float16")
    model.compile(optimizer=optimizer)
    model.fit(train_set, validation_data=validation_set, epochs=1)
    return model

In [ ]:
train()

In [4]:
model = layer.get_model("qa").get_train()

In [5]:
from transformers import AutoTokenizer
import tensorflow as tf
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
context = """
On a mild day in late July, a teenager was walking home from the downtown
area, such as it was, of Little Whinging. The city did not have a train station, a
department store, or even a large supermarket; just a few dozen shops, a bank, a
police station, and a library. It was the library from which soon-to-be-sixteen-yearold Harry Potter was returning, with an old bookbag around his shoulder. He
looked around from time to time as he walked. It looked as though he were
admiring the trees and bushes, which had recovered nicely from last year’s drought,
but he actually was wondering whether there was anyone following him. Or, more
precisely, whether he could catch a glimpse of the person he knew must be
following him. All he could see, however, were the normal sights of a suburban
neighborhood, and a few people looking at him rather oddly as they passed him.
Harry briefly wondered why–after all, he was not exactly famous in this area, nor
was his scar–until he realized that looking around to see if you were being followed
was not exactly usual behavior. 
"""

In [6]:
question = "When was the teenegar walking home?"
inputs = tokenizer([context], [question], return_tensors="np")
outputs = model(inputs)
start_position = tf.argmax(outputs.start_logits, axis=1)
end_position = tf.argmax(outputs.end_logits, axis=1)
print(int(start_position), int(end_position[0]))

In [7]:
answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]
print(answer)

In [8]:
print(tokenizer.decode(answer))